In [30]:
#Importing Packages
from functools import reduce
import pandas as pd
import warnings
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
warnings.filterwarnings('ignore')

In [2]:
df_user = pd.read_csv('Dataset/User_activity.txt')
df_user= df_user.transpose()
df_pub = pd.read_csv('Dataset/publishers.csv')
df_demo = pd.read_csv('Dataset/device_demographics.csv')

In [3]:
df_user.head(), df_pub.head(), df_demo.head()

(                     0                        1         2
 Unnamed: 0   device_id                 app_name   minutes
 0           B-52-23845    Snapchat (Mobile App)   27.9342
 1           B-52-16587   Pinterest (Mobile App)   14.6664
 2           B-52-36461    Facebook (Mobile App)  439.7652
 3           B-52-29089  Yahoo Mail (Mobile App)  900.6606,
                             app_name        Publisher
 0               Walmart (Mobile App)         Wal-Mart
 1        Microsoft Word (Mobile App)  Microsoft Sites
 2  Sam's Club Scan & Go (Mobile App)         Wal-Mart
 3             Pinterest (Mobile App)        Pinterest
 4               YouTube (Mobile App)     Google Sites,
     device_id  gender_id
 0  B-52-23845        2.0
 1  B-52-16587        1.0
 2  B-52-36461        2.0
 3  B-52-29089        2.0
 4  B-52-14212        2.0)

In [4]:
## Renaming Columns as First row
df_user1 = df_user.rename(columns=df_user.iloc[0]).drop(df_user.index[0])
df_user1.head()

,device_id,app_name,minutes
0,B-52-23845,Snapchat (Mobile App),27.9342
1,B-52-16587,Pinterest (Mobile App),14.6664
2,B-52-36461,Facebook (Mobile App),439.7652
3,B-52-29089,Yahoo Mail (Mobile App),900.6606
4,B-52-14212,Netflix (Mobile App),17.3502


In [5]:
# Dropping duplicates
df_user1.drop_duplicates(keep='first',inplace=True)
duplicate = df_user1[df_user1.duplicated(['device_id', 'app_name'])]
duplicate

,device_id,app_name,minutes


In [6]:
# Dropping duplicates
df_pub.drop_duplicates(keep='first',inplace=True)
duplicate = df_pub[df_pub.duplicated()]
duplicate

,app_name,Publisher


In [7]:
# Dropping duplicates
df_demo.drop_duplicates(keep='first',inplace=True)
duplicate = df_demo[df_demo.duplicated()]
duplicate

,device_id,gender_id


In [8]:
# Checking Null values
df_user1.isnull().sum(), df_pub.isnull().sum(), df_demo.isnull().sum()

(device_id      0
 app_name       0
 minutes      748
 dtype: int64,
 app_name     0
 Publisher    0
 dtype: int64,
 device_id     0
 gender_id    98
 dtype: int64)

In [9]:
df_user1.columns, df_pub.columns, df_demo.columns

(Index(['device_id', 'app_name', 'minutes'], dtype='object'),
 Index(['app_name', 'Publisher'], dtype='object'),
 Index(['device_id', 'gender_id'], dtype='object'))

In [10]:
df1 = pd.merge(df_user1,df_pub,on='app_name', how='outer')
df1.head()

,device_id,app_name,minutes,Publisher
0,B-52-23845,Snapchat (Mobile App),27.9342,"Snapchat, Inc"
1,B-52-6821,Snapchat (Mobile App),17.199,"Snapchat, Inc"
2,B-52-21945,Snapchat (Mobile App),181.7046,"Snapchat, Inc"
3,B-52-24149,Snapchat (Mobile App),133.3962,"Snapchat, Inc"
4,B-52-28462,Snapchat (Mobile App),1252.7676,"Snapchat, Inc"


In [11]:
df2 = pd.merge(df1, df_demo, on='device_id', how='outer')
df2.head()

,device_id,app_name,minutes,Publisher,gender_id
0,B-52-23845,Snapchat (Mobile App),27.9342,"Snapchat, Inc",2.0
1,B-52-23845,Pinterest (Mobile App),5.5188,Pinterest,2.0
2,B-52-23845,Instagram (Mobile App),867.132,Facebook,2.0
3,B-52-23845,Google Search (Mobile App),236.817,Google Sites,2.0
4,B-52-23845,YouTube (Mobile App),69.6276,Google Sites,2.0


In [12]:
df2.isnull().sum()

device_id      0
app_name       0
minutes      748
Publisher      0
gender_id    795
dtype: int64

In [13]:
## Checking Duplicates
df2[df2.duplicated()]

,device_id,app_name,minutes,Publisher,gender_id


In [14]:
## Sorting based on appname, minutes
df2.sort_values(by=['app_name','minutes'], inplace = True)
df2.head()

,device_id,app_name,minutes,Publisher,gender_id
18187,B-52-52155,Amazon Mobile (Mobile App),0.0378,Amazon Sites,2.0
5371,B-52-38817,Amazon Mobile (Mobile App),0.0756,Amazon Sites,2.0
10187,B-52-9082,Amazon Mobile (Mobile App),0.0756,Amazon Sites,2.0
19101,B-52-20729,Amazon Mobile (Mobile App),0.0756,Amazon Sites,1.0
5599,B-52-41002,Amazon Mobile (Mobile App),0.1134,Amazon Sites,1.0


In [15]:
## lookup table for GenderId for imputation
df3 = df2.groupby(['app_name','Publisher'])['gender_id'].agg(pd.Series.mode).reset_index(name = "gender_id")
df3.head()

,app_name,Publisher,gender_id
0,Amazon Mobile (Mobile App),Amazon Sites,2.0
1,Amazon Music with Prime Music (Mobile App),Amazon Sites,2.0
2,Facebook (Mobile App),Facebook,2.0
3,Facebook Messenger (Mobile App),Facebook,2.0
4,Google Play (Mobile App),Google Sites,2.0


In [16]:
df4 = pd.merge(df3, df2, how='left', on=['app_name','Publisher'])
df4.head()

,app_name,Publisher,gender_id_x,device_id,minutes,gender_id_y
0,Amazon Mobile (Mobile App),Amazon Sites,2.0,B-52-52155,0.0378,2.0
1,Amazon Mobile (Mobile App),Amazon Sites,2.0,B-52-38817,0.0756,2.0
2,Amazon Mobile (Mobile App),Amazon Sites,2.0,B-52-9082,0.0756,2.0
3,Amazon Mobile (Mobile App),Amazon Sites,2.0,B-52-20729,0.0756,1.0
4,Amazon Mobile (Mobile App),Amazon Sites,2.0,B-52-41002,0.1134,1.0


In [17]:
# replacing "Null" values with Mode of gender Id
df4['gender_id'] = df4['gender_id_y'].fillna(df4['gender_id_x'])
df4 = df4.drop(['gender_id_x','gender_id_y'], axis=1)
df4.head()

,app_name,Publisher,device_id,minutes,gender_id
0,Amazon Mobile (Mobile App),Amazon Sites,B-52-52155,0.0378,2.0
1,Amazon Mobile (Mobile App),Amazon Sites,B-52-38817,0.0756,2.0
2,Amazon Mobile (Mobile App),Amazon Sites,B-52-9082,0.0756,2.0
3,Amazon Mobile (Mobile App),Amazon Sites,B-52-20729,0.0756,1.0
4,Amazon Mobile (Mobile App),Amazon Sites,B-52-41002,0.1134,1.0


In [18]:
df4.isnull().sum()

app_name       0
Publisher      0
device_id      0
minutes      748
gender_id      0
dtype: int64

In [19]:
## lookup table for Minutes
df5 = df4.groupby(['app_name','Publisher'])['minutes'].agg(pd.Series.median).reset_index(name = "minutes")
df5.head()

,app_name,Publisher,minutes
0,Amazon Mobile (Mobile App),Amazon Sites,47.1555
1,Amazon Music with Prime Music (Mobile App),Amazon Sites,34.3980
2,Facebook (Mobile App),Facebook,2764.6542
3,Facebook Messenger (Mobile App),Facebook,1349.6490
4,Google Play (Mobile App),Google Sites,57.6639


In [97]:
df6 = pd.merge(df4, df5, how='left', on=['app_name','Publisher'])
df6.head()

,app_name,Publisher,device_id,minutes_x,gender_id,minutes_y
0,Amazon Mobile (Mobile App),Amazon Sites,B-52-52155,0.0378,2.0,47.1555
1,Amazon Mobile (Mobile App),Amazon Sites,B-52-38817,0.0756,2.0,47.1555
2,Amazon Mobile (Mobile App),Amazon Sites,B-52-9082,0.0756,2.0,47.1555
3,Amazon Mobile (Mobile App),Amazon Sites,B-52-20729,0.0756,1.0,47.1555
4,Amazon Mobile (Mobile App),Amazon Sites,B-52-41002,0.1134,1.0,47.1555


In [98]:
## Replacing null values with Median
df6['minutes'] = df6['minutes_x'].fillna(df6['minutes_y'])
df6 = df6.drop(['minutes_x','minutes_y'], axis=1)
df6.head()

,app_name,Publisher,device_id,gender_id,minutes
0,Amazon Mobile (Mobile App),Amazon Sites,B-52-52155,2.0,0.0378
1,Amazon Mobile (Mobile App),Amazon Sites,B-52-38817,2.0,0.0756
2,Amazon Mobile (Mobile App),Amazon Sites,B-52-9082,2.0,0.0756
3,Amazon Mobile (Mobile App),Amazon Sites,B-52-20729,1.0,0.0756
4,Amazon Mobile (Mobile App),Amazon Sites,B-52-41002,1.0,0.1134


In [106]:
df6["minutes"]=df6["minutes"].astype(float)
df6_grp = df6.groupby('app_name')['minutes'].agg(['median', 'std'])
df6_grp['thres'] = df6_grp['std'] + df6_grp['median']
df6_grp.loc['Amazon Mobile (Mobile App)','thres']

358.37869617823327

In [107]:
data_list = [row for row in df6.itertuples() if (float(row[5]) <= df6_grp.loc[row[1],'thres'] and float(row[5]) > 30.0)]

In [108]:
df7 = pd.DataFrame(data_list).set_index('Index')
df7.head()

,app_name,Publisher,device_id,gender_id,minutes
Index,,,,,
155,Amazon Mobile (Mobile App),Amazon Sites,B-52-36955,1.0,100.3590
156,Amazon Mobile (Mobile App),Amazon Sites,B-52-32471,2.0,101.3418
157,Amazon Mobile (Mobile App),Amazon Sites,B-52-10545,2.0,101.5308
158,Amazon Mobile (Mobile App),Amazon Sites,B-52-9595,2.0,101.6064
159,Amazon Mobile (Mobile App),Amazon Sites,B-52-3211,2.0,101.7198


In [115]:
df8 = df7.groupby('app_name')['minutes'].describe()
df8

,count,mean,std,min,25%,50%,75%,max
app_name,,,,,,,,
Amazon Mobile (Mobile App),828.0,119.505818,84.114884,30.0888,49.28175,86.5809,167.35950,357.8526
Amazon Music with Prime Music (Mobile App),126.0,147.709200,122.798296,32.2812,53.97840,98.0343,210.82950,477.7164
Facebook (Mobile App),1910.0,2421.228930,1683.694726,30.6180,926.48745,2307.0852,3681.78615,5995.2312
Facebook Messenger (Mobile App),1912.0,1396.553956,1138.905491,30.6180,428.68980,1172.6883,2101.08465,4386.3120
Google Play (Mobile App),1844.0,127.655345,109.966016,30.0132,53.51535,86.2407,162.72900,601.7382
Google Search (Mobile App),1747.0,345.808226,365.273923,30.0132,106.27470,219.7314,432.52650,1990.3968
GroupMe (Mobile App),92.0,131.112587,86.115822,30.5424,57.68280,102.9672,190.40805,329.9940
Instagram (Mobile App),1092.0,465.368365,447.561341,30.0132,111.48165,252.4473,728.84070,1724.3226
Kindle (Mobile App),129.0,344.901265,397.408270,30.9204,51.25680,168.3990,482.89500,1641.2382


In [117]:
df9 = df7.groupby(['app_name','gender_id'])['minutes'].describe()
df9

count         mean  \
app_name                                   gender_id                        
Amazon Mobile (Mobile App)                 1.0         304.0   121.036843   
                                           2.0         524.0   118.617590   
Amazon Music with Prime Music (Mobile App) 1.0          37.0   158.794735   
                                           2.0          89.0   143.100607   
Facebook (Mobile App)                      1.0         718.0  2453.191624   
                                           2.0        1192.0  2401.976233   
Facebook Messenger (Mobile App)            1.0         683.0  1415.977927   
                                           2.0        1229.0  1385.759349   
Google Play (Mobile App)                   1.0         682.0   128.698192   
                                           2.0        1162.0   127.043279   
Google Search (Mobile App)                 1.0         637.0   359.079587   
                                           2.0        1110.0   338.192139   
GroupMe (Mobile App)                       1.0          26.0   125.956869   
                                           2.0          66.0   133.143627   
Instagram (Mobile App)                     1.0         395.0   445.352710   
                                           2.0         697.0   476.711527   
Kindle (Mobile App)                        1.0          54.0   402.484600   
                                           2.0          75.0   303.441264   
Microsoft Word (Mobile App)                1.0           9.0    44.755200   
                                           2.0          27.0    51.924600   
Netflix (Mobile App)                       1.0         161.0   447.732313   
                                           2.0         272.0   463.943164   
Outlook (Mobile App)                       1.0          64.0   202.636941   
                                           2.0         147.0   192.706971   
Pinterest (Mobile App)                     1.0         162.0   239.335833   
                                           2.0         371.0   199.678245   
Sam's Club (Mobile App)                    1.0           1.0    58.098600   
                                           2.0           1.0    42.071400   
Sam's Club Scan & Go (Mobile App)          1.0           3.0    46.279800   
                                           2.0           4.0    39.642750   
Snapchat (Mobile App)                      1.0         287.0   365.329493   
                                           2.0         536.0   357.123610   
Tumblr (Mobile App)                        1.0          60.0   374.006430   
                                           2.0          90.0   647.080560   
Walmart (Mobile App)                       1.0          93.0    67.426461   
                                           2.0         208.0    72.139846   
Yahoo Mail (Mobile App)                    1.0         144.0   259.969500   
                                           2.0         252.0   241.572000   
Yahoo Newsroom (Mobile App)                1.0          12.0   117.009900   
                                           2.0          33.0   137.684782   
YouTube (Mobile App)                       1.0         652.0   535.578170   
                                           2.0        1102.0   510.484266   

                                                              std      min  \
app_name                                   gender_id                         
Amazon Mobile (Mobile App)                 1.0          84.443771  30.2022   
                                           2.0          83.991499  30.0888   
Amazon Music with Prime Music (Mobile App) 1.0         121.137675  33.2262   
                                           2.0         123.868258  32.2812   
Facebook (Mobile App)                      1.0        1705.788064  34.9272   
                                           2.0        1670.670339  30.6180   
Facebook Messenger (Mobile App)           

In [119]:
# Writing into CSV Files
df8.to_csv('usage_statastical_info_based_on_app_name.csv')
df9.to_csv('usage_statastical_info_based_on_app_name_and_gender_id.csv')

In [140]:
df10= df7.groupby('app_name')['minutes'].agg(('sum','count','mean')).reset_index()
df10.rename(columns={'sum':'total_minutes','count':'total_devices','mean':'Avg_time_spend_per_device'}, inplace=True)
df10

,app_name,total_minutes,total_devices,Avg_time_spend_per_device
0,Amazon Mobile (Mobile App),9.895082e+04,828,119.505818
1,Amazon Music with Prime Music (Mobile App),1.861136e+04,126,147.709200
2,Facebook (Mobile App),4.624547e+06,1910,2421.228930
3,Facebook Messenger (Mobile App),2.670211e+06,1912,1396.553956
4,Google Play (Mobile App),2.353965e+05,1844,127.655345
5,Google Search (Mobile App),6.041270e+05,1747,345.808226
6,GroupMe (Mobile App),1.206236e+04,92,131.112587
7,Instagram (Mobile App),5.081823e+05,1092,465.368365
8,Kindle (Mobile App),4.449226e+04,129,344.901265
9,Microsoft Word (Mobile App),1.804761e+03,36,50.132250


In [144]:
df10['Rank (user based)']= df10['total_devices'].rank(ascending=False)
df10['Rank (Duration based)'] = df10['total_minutes'].rank(ascending=False)
df10

,app_name,total_minutes,total_devices,Avg_time_spend_per_device,Rank (user based),Rank (Duration based)
0,Amazon Mobile (Mobile App),9.895082e+04,828,119.505818,7.0,10.0
1,Amazon Music with Prime Music (Mobile App),1.861136e+04,126,147.709200,16.0,16.0
2,Facebook (Mobile App),4.624547e+06,1910,2421.228930,2.0,1.0
3,Facebook Messenger (Mobile App),2.670211e+06,1912,1396.553956,1.0,2.0
4,Google Play (Mobile App),2.353965e+05,1844,127.655345,3.0,7.0
5,Google Search (Mobile App),6.041270e+05,1747,345.808226,5.0,4.0
6,GroupMe (Mobile App),1.206236e+04,92,131.112587,17.0,17.0
7,Instagram (Mobile App),5.081823e+05,1092,465.368365,6.0,5.0
8,Kindle (Mobile App),4.449226e+04,129,344.901265,15.0,13.0
9,Microsoft Word (Mobile App),1.804761e+03,36,50.132250,19.0,19.0


In [145]:
df10.to_csv('ranking_info_based_on_appname.csv')

In [191]:
df10['Publisher'] = df10['app_name'].apply(lambda x: df_pub.loc[df_pub[df_pub['app_name']== x].index[0], 'Publisher'])
df10

,app_name,total_minutes,total_devices,Avg_time_spend_per_device,Rank (user based),Rank (Duration based),Publisher
0,Amazon Mobile (Mobile App),9.895082e+04,828,119.505818,7.0,10.0,Amazon Sites
1,Amazon Music with Prime Music (Mobile App),1.861136e+04,126,147.709200,16.0,16.0,Amazon Sites
2,Facebook (Mobile App),4.624547e+06,1910,2421.228930,2.0,1.0,Facebook
3,Facebook Messenger (Mobile App),2.670211e+06,1912,1396.553956,1.0,2.0,Facebook
4,Google Play (Mobile App),2.353965e+05,1844,127.655345,3.0,7.0,Google Sites
5,Google Search (Mobile App),6.041270e+05,1747,345.808226,5.0,4.0,Google Sites
6,GroupMe (Mobile App),1.206236e+04,92,131.112587,17.0,17.0,Microsoft Sites
7,Instagram (Mobile App),5.081823e+05,1092,465.368365,6.0,5.0,Facebook
8,Kindle (Mobile App),4.449226e+04,129,344.901265,15.0,13.0,Amazon Sites
9,Microsoft Word (Mobile App),1.804761e+03,36,50.132250,19.0,19.0,Microsoft Sites


In [193]:
# Changing Columns Order
column_names = ['app_name','Publisher', 'total_minutes', 'total_devices','Avg_time_spend_per_device', 'Rank (user based)','Rank (Duration based)']
df11 = df10[column_names]
df11.columns

Index(['app_name', 'Publisher', 'total_minutes', 'total_devices',
       'Avg_time_spend_per_device', 'Rank (user based)',
       'Rank (Duration based)'],
      dtype='object')

In [194]:
df11.head()

,app_name,Publisher,total_minutes,total_devices,Avg_time_spend_per_device,Rank (user based),Rank (Duration based)
0,Amazon Mobile (Mobile App),Amazon Sites,9.895082e+04,828,119.505818,7.0,10.0
1,Amazon Music with Prime Music (Mobile App),Amazon Sites,1.861136e+04,126,147.709200,16.0,16.0
2,Facebook (Mobile App),Facebook,4.624547e+06,1910,2421.228930,2.0,1.0
3,Facebook Messenger (Mobile App),Facebook,2.670211e+06,1912,1396.553956,1.0,2.0
4,Google Play (Mobile App),Google Sites,2.353965e+05,1844,127.655345,3.0,7.0


In [195]:
df11.to_csv('ranking_info_based_on_appname_publisher.csv')